# SYN_BURST protocol testing

This is the minimal setup to trigger a burst using a combination of GLU + GABA synapses in the full Gillies-Willshaw STN neuron model

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook
# Enable connecting with ipyton console --existing
# %connect_info

## Import all our modules

In [ ]:
# Fix NEURON stdout output
import sys
default_stdout = sys.stdout
default_stderr = sys.stderr

# NEURON modules
import neuron
from neuron import h
h.load_file("stdlib.hoc") # Load the standard library
h.load_file("stdrun.hoc") # Load the standard run library

# Add our own modules to Python path
import os.path
scriptdir = os.path.abspath('.')
modulesbase = os.path.normpath(os.path.join(scriptdir, '..'))
sys.path.append(modulesbase)

# Gillies-Willshaw STN model
import gillies_model as gillies

# Plotting & recording
from bgcellmodels.common import analysis

# Physiological parameters
import bgcellmodels.cellpopdata as cpd
from bgcellmodels.cellpopdata import PhysioState, Populations as Pop, NTReceptors as NTR, ParameterSource as Cit

# STN cell testing
from stn_model_evaluation import *

# Fix NEURON output
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = default_stdout
sys.stderr = default_stderr

## Build STN model
Build STN cell and evaluator object

In [ ]:
# Make cell model and evaluator
evaluator = StnModelEvaluator(StnModel.Gillies2005, PhysioState.NORMAL)
evaluator.build_cell(StnModel.Gillies2005)

proto_GABA = StimProtocol.SINGLE_SYN_GABA
proto_GLU = StimProtocol.SINGLE_SYN_GLU

## Simulate multiple synapses

Increase the number of synapses until we get a rebound burst

### Eliciting rebound burst

Rebound burst protocol: (Gillies 2005, Fig. 3-4):

- induced with 500ms -025 nA somatic current injection
- Fig. 3D + 4C-D: hyperpolarized to between -82 ~ -75 mV
- increasing g_CaL or decreasing g_sKCa increased burst duration


Plateau potential protocol: (Gillies 2005, Fig. 10C-D)

- hyperpolarize to -70 mV (below thresold CaT deinactivation, i.e. CaT will be triggered) (see fig. 10C)
- +0.2 nA depolarizing pulse (see fig. 10D) over hyperpolarizing current of duration 50 ms (see fig. 10D, top right)

In [ ]:
# Change SKCa conductance
for sec in h.allsec():
    for seg in sec:
        seg.gk_sKCa = 0.6 * seg.gk_sKCa

# Add some synapses
evaluator.make_GABA_inputs(1)
evaluator.make_GLU_inputs(4)

evaluator.rec_traces(proto_GABA)
evaluator.rec_traces(proto_GLU)

In [ ]:
# Print synapse list
h.topology() # see console from which Jupyter was started
print("List of synapses:")
all_syns = sum((evaluator.model_data[evaluator.target_model]['inputs'][pop]['synapses'] for pop in ['gpe', 'ctx']),[])
for syn in all_syns:
    print("Synapse {} in segment {}".format(syn, syn.get_segment()))

In [ ]:
# Simulate with additional synapses
# h.cvode_active(0)
evaluator.init_sim()
evaluator.run_sim()

In [ ]:
# Plot simulation data
evaluator.plot_traces(proto_GABA)
evaluator.plot_traces(proto_GLU)